The easiest way to predict whether a problem is a classification or regression problem is by checking the continuity of the output given.In cases where there is a ontinous output,it is a regression problem and vice versa.

For the given diabetes dataset:
Threshold is:Glucose,BMI and Pregnancies.
The means of the three values are taken and values below the man are branded as a 0.Those above are 1.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# Load your CSV
data = pd.read_csv('diabetes.csv')

# Standardize features like scikit-learn dataset
def standardize(x):
   return (x - x.mean()) / x.std()

# Create matching features
processed_data = pd.DataFrame({
   'age': standardize(data['Age']),
   'sex': standardize(data['Pregnancies']),  # Using pregnancies as proxy for sex
   'bmi': standardize(data['BMI']),
   'bp': standardize(data['BloodPressure']),
   's1': standardize(data['SkinThickness']), 
   's2': standardize(data['Glucose']),
   's3': standardize(data['DiabetesPedigreeFunction']),
   's4': standardize(data['Insulin']),
   's5': standardize(data['BMI'] * data['Age'] / 100),  # Creating derived feature
   's6': standardize(data['DiabetesPedigreeFunction'] * data['Age'] / 100)  # Creating derived feature
})

processed_data = processed_data[['s2', 'bmi', 'sex']]
processed_data['actual_value']=data['Outcome']


In [19]:
print(processed_data)

           s2       bmi       sex  actual_value
0    0.847771  0.203880  0.639530             1
1   -1.122665 -0.683976 -0.844335             0
2    1.942458 -1.102537  1.233077             1
3   -0.997558 -0.493721 -0.844335             0
4    0.503727  1.408828 -1.141108             1
..        ...       ...       ...           ...
763 -0.622237  0.115094  1.826623             0
764  0.034575  0.609757 -0.547562             0
765  0.003299 -0.734711  0.342757             0
766  0.159683 -0.240048 -0.844335             1
767 -0.872451 -0.201997 -0.844335             0

[768 rows x 4 columns]


In [20]:
def normalization(x):
    return (x-min(x))/(max(x)-min(x))

In [21]:
processed_data['bmi']=normalization(processed_data['bmi'])
processed_data['bmi'] = (processed_data['bmi'] >= processed_data['bmi'].mean()).astype(int)


In [22]:
def normalize_and_binarize(dataframe, columns):
    for column in columns:
        dataframe[column] = normalization(dataframe[column])
        dataframe[column] = (dataframe[column] >= dataframe[column].mean()).astype(int)
    
    return dataframe

columns_to_process = ['bmi', 's2', 'sex'] 
processed_data = normalize_and_binarize(processed_data, columns_to_process)

In [23]:
processed_data['pred_value']=((processed_data['s2']==processed_data['bmi'])
                               &(processed_data['s2']==processed_data['sex'])).astype(int)

processed_data=processed_data[['pred_value','actual_value']]

There are four classifications we need to group the data into from this moment on:
True Positive(TP)-When actual_value is 1 and pred_value is 1
True Negative(TN)-actual_value is 0 and pred_ value is 0
False Positive(FP)-actual value is 0 and pred_value is 1
False Negative(FN)-actual_value is 1 and pred_value is 0

In [24]:
processed_data['actual_value'].value_counts()

actual_value
0    500
1    268
Name: count, dtype: int64

In [25]:
processed_data['pred_value'].value_counts()

pred_value
0    507
1    261
Name: count, dtype: int64

In [26]:
processed_data['TP']=((processed_data['actual_value']==1)
                      & (processed_data['pred_value']==1)).astype(int)
processed_data['TN']=((processed_data['actual_value']==0)
                      & (processed_data['pred_value']==0)).astype(int)
processed_data['FP']=((processed_data['actual_value']==0)
                      & (processed_data['pred_value']==1)).astype(int)
processed_data['FN']=((processed_data['actual_value']==1)
                      & (processed_data['pred_value']==0)).astype(int)

In [27]:
print(processed_data)

     pred_value  actual_value  TP  TN  FP  FN
0             1             1   1   0   0   0
1             1             0   0   0   1   0
2             0             1   0   0   0   1
3             1             0   0   0   1   0
4             0             1   0   0   0   1
..          ...           ...  ..  ..  ..  ..
763           0             0   0   1   0   0
764           0             0   0   1   0   0
765           0             0   0   1   0   0
766           0             1   0   0   0   1
767           1             0   0   0   1   0

[768 rows x 6 columns]


Finding the valuecounts for each category

In [28]:
processed_data['TP'].value_counts()

TP
0    690
1     78
Name: count, dtype: int64

In [29]:
processed_data['TN'].value_counts()

TN
0    451
1    317
Name: count, dtype: int64

In [30]:
processed_data['FP'].value_counts()

FP
0    585
1    183
Name: count, dtype: int64

In [31]:
processed_data['FN'].value_counts()

FN
0    578
1    190
Name: count, dtype: int64

In [33]:
conf_matrix=[[78,183],[190,317]]

In [41]:
metrics={
    'Accuracy':395/768,
    'Precision':78/268,
    'Recall':78/261    
}
print(metrics)

{'Accuracy': 0.5143229166666666, 'Precision': 0.291044776119403, 'Recall': 0.2988505747126437}
